<a href="https://colab.research.google.com/github/cherieuu/Netflix_Analysis/blob/master/%EC%9C%A0%ED%99%94%EC%98%81/%EA%B8%B8%ED%98%B8%EB%8B%98_%ED%83%9C%EA%B7%B8_viewpoint_%EB%B6%84%EC%84%9D_%EC%8B%9C%EA%B0%81%ED%99%94%20%EC%99%84%EB%A3%8C_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/PJT6-Netflix/Netflix_Analysis.git

# Tag_viewpoint 분석

In [ ]:
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from wordcloud import WordCloud

# 가설검정
from scipy import stats

# 경고문구 무시
import warnings
warnings.filterwarnings("ignore")
# 자연어 처리
# import spacy

## 데이터 정보확인

In [ ]:
df = pd.read_csv('/content/Netflix_Analysis/use_df.csv')

In [ ]:
df.head(2)

In [ ]:
df.describe()

In [ ]:
colors = ['black',] * 2
colors[0] = 'crimson'
count = df['Series or Movie'].value_counts()

In [ ]:
# 데이터 시각화 plot.graph_objects 사용
fig = go.Figure(data=[go.Bar(
    x = df["Series or Movie"],
    y = count,
    text = count,
    textposition='auto',
    marker_color=colors 
)])

In [ ]:
# 바차트 제목 및 y축 라벨, x축 라벨 그리기
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'More Movie or Tv Series?',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size= 14),
                  xaxis=dict(
                  title='Category',
                  titlefont_size= 14)
                  )

## viewpoint 정규화
- min-max 정규화 실시

In [ ]:
df['VALUE_std'] = (df['VALUE'] - df['VALUE'].min())/df['VALUE'].max()

- 표준화 실시

In [ ]:
df['VALUE_norm'] = (df['VALUE'] -df['VALUE'].mean())/df['VALUE'].std()

## Tag_count 변수 추가

In [ ]:
len(str(df.Tags.iloc[0]).split(','))

In [ ]:
rows = len(df['Tags'])

In [ ]:
df['Tags'].iloc[0]

In [ ]:
df['Tag_count'] = 0

In [ ]:
for i in range(rows):
    df.Tag_count[i] = len(str(df.Tags.iloc[i]).split(','))

In [ ]:
df.head(1)

In [ ]:
# 정규화 진행

df['Tag_count_norm'] = (df['Tag_count'] - df['Tag_count'].min())/df['Tag_count'].max()  ## 

## movie 데이터와 series 데이터 분리

In [ ]:
df_movie = df[df['Series or Movie']=='Movie']
df_series = df[df["Series or Movie"] == "Series"]

## movie 태그 수와 viewpoint
- 데이터 정리 및 카이제곱 검정(독립성 검사)

In [ ]:
df_movie.head(1)

In [ ]:
df_movie[['Title','Tags','VALUE','Tag_count']]

In [ ]:
df_count = df_movie[['VALUE','Tag_count']]

In [ ]:
df_count.isnull().sum()

In [ ]:
df_count.head(10)

In [ ]:
len(df_count['Tag_count'].unique())

In [ ]:
df_count.describe()

### 카이제곱검정 
- 귀무가설 : 두 변수는 연관성이 없다, 차이가 없다. 
    - => tag_count의 증감은 value와의 상관성이 없다 
- 대립가설 : 두 변수는 연관성이 있다, 차이가 있다
    - => tag_count의 증감은 value와의 상관성이 있다

In [ ]:
value = list(df_count.VALUE)
tag_count = list(df_count.Tag_count)

In [ ]:
chis= stats.chisquare(value, tag_count)

In [ ]:
statistic,pvalue = chis
print("static : {}, p-value : {}".format(statistic,pvalue))
if pvalue < 0.05:
    print("신뢰수준하에서 유의한 관계. 대립가설 채택")
else:
    print("신뢰수준하에서 무의미한 관계. 귀무가설 채택")

In [ ]:
df_count.corr()

In [ ]:
plt.scatter(value, tag_count)
plt.show()

 -  카이제곱검정 결과 두 변수들간의 유의수준 0.05에서 tag_count의 증감은 value와의 상관성이 있다고 할 수 있다.
 - 산점도와 상관계수 확인을 해봤을 때 약한 양의 상관성이 있다고 볼 수 있다.

## movie 태그들과 viewpoint
 - 귀무가설 : 태그들은 viewpoint에 영향이 없다
 - 대립가설 : 태그들은 viewpoint에 영향이 있다

### 태그 분할
- 태그들과 뷰포인트 부분

In [ ]:
df_tag = df_movie[['VALUE','Tags']]

In [ ]:
df_tag.head(1)

In [ ]:
df_tag.info()

In [ ]:
df_tag.sort_values('VALUE',ascending = False,inplace = True)

In [ ]:
# viewpoint 기준으로 상위 20개의 영화를 가져옴
df_view_top20 = df_tag[:][:20]

#### viewpoint 기준 top20 tag들을 single tag로 만듬

In [ ]:
df_view_top20.set_index('VALUE',inplace=True)

In [ ]:
df_view_top20.head(10)

In [ ]:
result = df_view_top20['Tags'].str.split(',')  # 리스트화

In [ ]:
result = result.apply(lambda x: pd.Series(x))

In [ ]:
# result.stack()
# result.stack().reset_index(level=1, drop=True) ## 인덱스 변화 확인

In [ ]:
result = result.stack().reset_index(level=1, drop=True).to_frame('Tag_single')
result

In [ ]:
rs = result.reset_index()

In [ ]:
rs.head(1)

In [ ]:
rs.count

In [ ]:
len(rs['Tag_single'].unique())  # top20의 태그들 종류수

### 전체 태그별 카운트 수 확인 및 빈도 수를 워드클라우드로 시각화


In [ ]:
from collections import Counter

In [ ]:
# 전체 데이터셋에서 리스트 태그값을 싱글 태그로 변환
df_view = df_tag.set_index('VALUE')
result_All_tag = df_view['Tags'].str.split(',')
result_All_tag = result_All_tag.apply(lambda x: pd.Series(x))
result_All_tag = result_All_tag.stack().reset_index(level=1, drop=True).to_frame('Tag_single')
result_All_tag = result_All_tag.reset_index()

In [ ]:
tag_counter = Counter(result_All_tag.Tag_single)

In [ ]:
wordcloud = WordCloud(width = 1400, height = 1400, # 단어별 빈도수로 워드 클라우드 작성 
                background_color ='lavenderblush',  
                min_font_size = 5).generate_from_frequencies(tag_counter)

In [ ]:
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

## 태그들의 빈도수가 뷰포인트와 상관성이 있을까

뷰포인트 상위 20개의 태그들의 빈도수와 뷰포인트 간의 상관성 시각화

In [ ]:
# 전체 데이터셋에서 태그 개수
tag_counter = dict(tag_counter)

In [ ]:
dfdf = pd.DataFrame(pd.Series(tag_counter), columns=['counter']) 

In [ ]:
dfdf.head(1)

In [ ]:
# 카운트수를 기준으로 내림차순
dfdf.sort_values(by = 'counter',ascending = False,inplace = True)

In [ ]:
dfdf = dfdf.reset_index()

In [ ]:
dfdf.columns = ['Tag','counter']

In [ ]:
# 전체 데이터셋에서 상위 20개의 태그 수
dfdf_top20 = dfdf[:][:20]

In [ ]:
dfdf_top20.head(1)

In [ ]:
from matplotlib import gridspec

# 전체 데이터 셋에서 top 20 태그들의 분포

fig = plt.figure(figsize=(20, 6))
gs = gridspec.GridSpec(nrows=1, ncols=2,
                       height_ratios=[6], 
                       width_ratios=[10, 5])

ax = plt.subplot(gs[0])
sns.barplot(dfdf_top20['Tag'], dfdf_top20['counter'], ax=ax, palette="RdGy")
ax.set_xticklabels(dfdf_top20['Tag'], rotation='90')
ax.set_title('Top 20 Tag', fontsize=15, fontweight='bold')

explode = [0 for _ in range(20)]
explode[0] = 0.06

ax2 = plt.subplot(gs[1])
ax2.pie(dfdf_top20['counter'], labels=dfdf_top20['Tag'],
        shadow=True, startangle=0, explode=explode,
        colors=sns.color_palette("RdGy", n_colors=20)
       )
ax2.axis('equal') 

plt.show()

In [ ]:
# top20 뷰포인트의 태그값의 개수
view_counter = Counter(rs.Tag_single)

In [ ]:
view_dfdf = pd.DataFrame(pd.Series(view_counter), columns=['counter']) 

In [ ]:
view_dfdf.sort_values(by = 'counter',ascending = False,inplace = True)  # 내림차순 정렬
view_dfdf = view_dfdf.reset_index()
view_dfdf.columns = ['Tag','counter']

In [ ]:
view_dfdf['Tag'].count()

In [ ]:
# 뷰포인트 기준 top 20 태그들의 분포

fig = plt.figure(figsize=(20, 6))
gs = gridspec.GridSpec(nrows=1, ncols=2,
                       height_ratios=[6], 
                       width_ratios=[10, 5])

ax = plt.subplot(gs[0])
sns.barplot(view_dfdf['Tag'], view_dfdf['counter'], ax=ax, palette="RdGy")
ax.set_xticklabels(view_dfdf['Tag'], rotation='90')
ax.set_title('Top 20 Viewpoint Tag', fontsize=15, fontweight='bold')

explode = [0 for _ in range(view_dfdf['Tag'].count())]
explode[0] = 0.06

ax2 = plt.subplot(gs[1])
ax2.pie(view_dfdf['counter'], labels=view_dfdf['Tag'],
        shadow=True, startangle=0, explode=explode,
        colors=sns.color_palette("RdGy", n_colors=view_dfdf['Tag'].count())
       )
ax2.axis('equal') 

plt.show()

**얼마나 비슷할까?**

In [ ]:
top_count = dfdf_top20['Tag'].count()

In [ ]:
check = list(view_dfdf['Tag'].unique())

In [ ]:
check_list = []

In [ ]:
for i in range(top_count):
    if dfdf_top20['Tag'][i] in check :
        check_list.append(1)
    else:
        check_list.append(0)

In [ ]:
print(sum(check_list)/len(check_list))

## 결론
- movie 유형에서 한 영화에 들어있는 태그 수와 viewpoint간의 독립성 검정을 위해 카이제곱검정을 실시했는 데 서로 연관이 있는 것으로 나왔다.
- 상식적으로 한 영화에 들어있는 태그 종류 수가 많으면 검색결과에서 출현빈도가 높아지므로 한 영화에 들어있는 태그 종류 수가 많을 수록 viewpoint가 높을 것으로 예상된다.

- 또한 상위 태그들이 viewpoint가 높은 영화에 얼마나 많이 출현하는 지 확인 하기 위해서 전체 데이터셋에서 각 태그들이 나타나는 빈도를 확인하였고, 상위 20개의 태그들의 빈도를 시각화하여 어떤 태그들이 많이 나타나는지 확인하였다.
- 또한, viewpoint기준 상위 20개의 영화의 태그들의 빈도 수를 확인하여 시각화하였고, viewpoint기준 상위 20개의 영화 태그들과 전체 데이터셋의 태그의 빈도 수를 비교해 본 결과 85%의 유사도를 확인 하였다.
- 결과적으로 태그가 많을 수록 특정 태그가 있을 수록 상위 viewpoint 영화와 연관이 있을 것으로 보인다.

## runtime 분석